In [1]:
%autosave 0
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Téléchargez la liste de stop words en français
nltk.download('stopwords')
nltk.download('punkt')

# Importez la liste de stop words en français
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df_test = pd.read_csv("Data/df_featEngin.csv")
df_test.head(10)

,index,Nombre_avis_publie,Note,Titres,Commentaire,Verifications,longCommentaire,longTitres,nb_Mots_Commentaire,nb_Mots_Titres,...,Titres_3grame_livraison_trop_longue,Titres_3grame_delai_livraison_trop,Titres_3grame_rapport_qualite_prix,Titres_3grame_livraison_rapide_conforme,Titres_3grame_livraison_peu_longue,Titres_3grame_livraison_rapide_prix,TextBlob_polarity_Commentaire,TextBlob_polarity_Titres,Saison_experience,Sentiment
0,0,1,5,Je commande mes pneus de motos et de voitures,Je commande mes pneus de motos et de voitures ...,1,215,45,31,9,...,0,0,0,0,0,0,0.271000,0.00,3,1
1,1,6,5,"Ne cherchez plus vos pneus, Allopneus le peut","BonjourLe site est genial, tres bien explique ...",1,315,45,48,8,...,0,0,0,0,0,0,0.400000,0.15,3,1
2,2,3,5,Tout etait parfait,"Tout etait parfait, timing parfait dans les da...",1,227,18,36,3,...,0,0,0,0,0,0,0.391667,0.51,3,1
3,3,4,5,Pneus d'un tres bon rapport qualite_,Pneus d'un tres bon rapport qualite prix. Les ...,1,157,36,22,6,...,0,0,0,0,0,0,0.607500,1.00,3,1
4,4,1,4,"Bon suivi de la commande,les questions_","Bon suivi de la commande,les questions sont im...",1,218,39,34,6,...,0,0,0,0,0,0,0.133333,0.70,3,1
5,5,1,5,"Excellent service, bien plus avantageux compar...",Ce service sur place est d un grand secours. R...,1,276,62,48,8,...,0,0,0,0,0,0,0.202083,0.21,3,1
6,6,1,5,commande jusqu'au montage,La commande sur internet c'est tres bien passe...,1,439,25,71,3,...,0,0,0,0,0,0,0.216923,0.00,3,1
7,7,4,5,Excellent,"RAS, le rapport qualite prix est tres correct....",1,217,9,39,1,...,0,0,0,0,0,0,0.367143,0.50,3,1
8,8,3,5,Fidele depuis de nombreuses annees j ai_,Fidele depuis de nombreuses annees j ai toujou...,1,112,40,19,7,...,0,0,0,0,0,0,0.282500,0.05,3,1
9,9,1,5,Meilleur rapport qualite prix,M'a ete recommande. Meilleur rapport qualite p...,1,183,29,30,4,...,0,0,1,0,0,0,0.207500,0.53,3,1


In [3]:
# (a) Supprimer les caractères spéciaux, chiffres, valeurs manquantes et mettre le texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Titres'] = df_test['Titres'].fillna('') 
#df_test = df_test.drop(columns=[])
df_test.head(10)

C:\Users\joan\AppData\Local\Temp\ipykernel_14132\2952274688.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
C:\Users\joan\AppData\Local\Temp\ipykernel_14132\2952274688.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Titres'] = df_test['Titres'].str.replace('[^a-zA-Z\s]', '').str.lower()


,index,Nombre_avis_publie,Note,Titres,Commentaire,Verifications,longCommentaire,longTitres,nb_Mots_Commentaire,nb_Mots_Titres,...,Titres_3grame_livraison_trop_longue,Titres_3grame_delai_livraison_trop,Titres_3grame_rapport_qualite_prix,Titres_3grame_livraison_rapide_conforme,Titres_3grame_livraison_peu_longue,Titres_3grame_livraison_rapide_prix,TextBlob_polarity_Commentaire,TextBlob_polarity_Titres,Saison_experience,Sentiment
0,0,1,5,je commande mes pneus de motos et de voitures,je commande mes pneus de motos et de voitures ...,1,215,45,31,9,...,0,0,0,0,0,0,0.271000,0.00,3,1
1,1,6,5,ne cherchez plus vos pneus allopneus le peut,bonjourle site est genial tres bien explique p...,1,315,45,48,8,...,0,0,0,0,0,0,0.400000,0.15,3,1
2,2,3,5,tout etait parfait,tout etait parfait timing parfait dans les dat...,1,227,18,36,3,...,0,0,0,0,0,0,0.391667,0.51,3,1
3,3,4,5,pneus dun tres bon rapport qualite,pneus dun tres bon rapport qualite prix les me...,1,157,36,22,6,...,0,0,0,0,0,0,0.607500,1.00,3,1
4,4,1,4,bon suivi de la commandeles questions,bon suivi de la commandeles questions sont imm...,1,218,39,34,6,...,0,0,0,0,0,0,0.133333,0.70,3,1
5,5,1,5,excellent service bien plus avantageux compare...,ce service sur place est d un grand secours ri...,1,276,62,48,8,...,0,0,0,0,0,0,0.202083,0.21,3,1
6,6,1,5,commande jusquau montage,la commande sur internet cest tres bien passee...,1,439,25,71,3,...,0,0,0,0,0,0,0.216923,0.00,3,1
7,7,4,5,excellent,ras le rapport qualite prix est tres correct t...,1,217,9,39,1,...,0,0,0,0,0,0,0.367143,0.50,3,1
8,8,3,5,fidele depuis de nombreuses annees j ai,fidele depuis de nombreuses annees j ai toujou...,1,112,40,19,7,...,0,0,0,0,0,0,0.282500,0.05,3,1
9,9,1,5,meilleur rapport qualite prix,ma ete recommande meilleur rapport qualite pri...,1,183,29,30,4,...,0,0,1,0,0,0,0.207500,0.53,3,1


In [4]:
from gensim.models import FastText

# Replace 'path/to/pretrained/embeddings' with the actual path to your downloaded embeddings file
model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")

C:\Users\joan\AppData\Local\Temp\ipykernel_14132\2105882537.py:4: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format("FastText/cc.fr.300.bin.gz")


In [5]:
# Define preprocessing function
stop_words = set(stopwords.words('french'))

def preprocess_text(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

# Apply FastText embeddings to each element in the Series
def get_text_embedding(text):
    preprocessed_text = preprocess_text(text)
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    
    if word_embeddings:
        text_embedding = np.mean(word_embeddings, axis=0)  # Use mean for aggregation
    else:
        text_embedding = np.zeros(model.vector_size)  # Use zero vector if no embeddings found
    
    return text_embedding


In [6]:
# Create a new column 'embedding' with the text embeddings
df_test['embedding_Commentaire'] = df_test['Commentaire'].apply(get_text_embedding)
df_test['embedding_Titres'] = df_test['Titres'].apply(get_text_embedding)

In [7]:
df_expanded = df_test['embedding_Commentaire'].apply(lambda x: pd.Series(x, dtype="float64")).add_prefix('vectComment_')
df_test = pd.concat([df_test, df_expanded], axis=1)
df_test = df_test.drop('embedding_Commentaire', axis=1)

df_expanded2 = df_test['embedding_Titres'].apply(lambda x: pd.Series(x, dtype="float64")).add_prefix('vectTitre_')
df_test = pd.concat([df_test, df_expanded2], axis=1)
df_test = df_test.drop('embedding_Titres', axis=1)


In [8]:
print(df_test.info())

#print(X_train.tolist()[0:5])
#print([x for x in X_train.apply(lambda x: len(x)) if x != 300])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285983 entries, 0 to 285982
Columns: 671 entries, index to vectTitre_299
dtypes: float64(602), int64(67), object(2)
memory usage: 1.4+ GB
None


In [9]:

df = df_test.filter(like = 'vect')
                    
df = pd.concat([df,df_test[['Nombre_avis_publie','longCommentaire','longTitres','nb_Mots_Commentaire','nb_Mots_Titres', 'nb_majuscules_Commentaire',
       'nb_chiffres_Commentaire', 'nb_ponctuation_Commentaire',
       'nb_special_Commentaire', 'nb_majuscules_Titre', 'nb_chiffres_Titre',
       'nb_ponctuation_Titre', 'nb_special_Titre', 'Sentiment']]],axis=1)
                                
                    
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285983 entries, 0 to 285982
Columns: 614 entries, vectComment_0 to Sentiment
dtypes: float64(600), int64(14)
memory usage: 1.3 GB
None
   vectComment_0  vectComment_1  vectComment_2  vectComment_3  vectComment_4  \
0       0.011306       0.016411      -0.017874      -0.011596      -0.065694   
1       0.019220       0.009953      -0.026793      -0.008516      -0.085727   
2       0.014238       0.001802      -0.016503       0.002777      -0.093255   
3       0.008320      -0.009746      -0.042637      -0.001757      -0.087079   
4       0.030223       0.006146      -0.021092      -0.012093      -0.062279   
5       0.014521       0.008975      -0.038061      -0.007624      -0.109386   
6       0.021570       0.008288      -0.027958       0.000285      -0.086603   
7       0.016658      -0.001761      -0.030290      -0.014614      -0.080752   
8      -0.004167       0.025591      -0.054842      -0.022558      -0.037973   
9       0.00960

In [10]:
# (b) Division du DataFrame en ensembles d'entraînement et de test.

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="Sentiment"), df['Sentiment'], test_size=0.2, random_state=42)

## Random Forest:

In [11]:
# (d) Création du classificateur RandomForest.
clf = make_pipeline(StandardScaler(with_mean=False), RandomForestClassifier(random_state=42))

In [12]:
# (e) Entraînement du classificateur sur l'ensemble d'entraînement.
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [13]:
# (f) Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

           0       0.84      0.69      0.76     10645
           1       0.93      0.97      0.95     46552

    accuracy                           0.92     57197
   macro avg       0.89      0.83      0.85     57197
weighted avg       0.91      0.92      0.91     57197



In [14]:
# (g) Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Classe prédite,0,1
Classe réelle,,
0,7366,3279
1,1412,45140


# XGBoost:

In [15]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [16]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(estimator=clf_xgb, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=...None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]})

In [17]:
best_params = grid_search.best_params_
best_clf_xgb = grid_search.best_estimator_
y_pred = best_clf_xgb.predict(X_test)

In [18]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9404339388429463

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.82      0.84     10645
           1       0.96      0.97      0.96     46552

    accuracy                           0.94     57197
   macro avg       0.91      0.89      0.90     57197
weighted avg       0.94      0.94      0.94     57197


Confusion Matrix:
 [[ 8712  1933]
 [ 1474 45078]]


# NEURAL NETWORK

## additionnal libraries

In [27]:
from keras import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras.layers import Input, Dense 

from tensorflow.keras.models import Model

In [28]:
inputs = Input(shape = X_train.shape[1], name = "Input")

dense1 = Dense(units = 20, activation = "tanh",kernel_initializer ='normal', name = "Dense_1")
dense2 = Dense(units = 1, activation = "sigmoid",kernel_initializer ='normal', name = "Dense_2")

x=dense1(inputs)

outputs=dense2(x)

model = Model(inputs = inputs, outputs = outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 613)]             0         
                                                                 
 Dense_1 (Dense)             (None, 20)                12280     
                                                                 
 Dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 12,301
Trainable params: 12,301
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

training_history = model.fit(X_train, y_train,          # données d'entraînement
                             epochs = 20,               # nombre d'epochs
                             batch_size = 200,          # taille des batchs
                             validation_split = 0.2)

train_acc = training_history.history['accuracy']
val_acc = training_history.history['val_accuracy']

NameError: name 'tensorflow' is not defined

In [ ]:
# Labels des axes
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Courbe de la précision sur l'échantillon d'entrainement
plt.plot(np.arange(1 , 21, 1),
         training_history.history['accuracy'],
         label = 'Training Accuracy',
         color = 'blue')

# Courbe de la précision sur l'échantillon de test
plt.plot(np.arange(1 , 21, 1),
         training_history.history['val_accuracy'], 
         label = 'Validation Accuracy',
         color = 'red')

# Affichage de la légende
plt.legend()

# Affichage de la figure
plt.show()

In [ ]:
# Prediction de l'échantillon de test
test_pred = model.predict(X_test)

# Evaluation du modèle
score = model.evaluate(X_test, y_test)

In [ ]:
print(classification_report(y_test, test_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_test_class, test_pred_class)
print(cnf_matrix)